# Tarea 2
A partir del corpus proporcionado realizar un modelo del lenguaje neuronal con base en la arquitectura de word2vec. Siganse los siguientes pasos:

1. Limpiar los Textos y apliccar stemming a las palabras. **[DONE]**
2. Insertar simbolos de inicio y final de cadena. **[DONE]**
3. Obtener los bigramas que aparecen en este texto. 
4. Entrenar con los bigramas la red neuronal y obtener los valores para los hiperparamteros. Tomar de 100 a 300 unidades para la capa oculta.
5. Obtener la matriz A y Pi a partir de las salidas de la red neuronal 
6. Calcular la probabilidad de las siguientes oraciones:
    - Nos bañamos con agua caliente 
    - El animalito le olía la cabeza
    - Pascuala ordeñaba las vacas

In [35]:
# Importamos las librerias

# Usamos os para cargar los libros
from os import listdir,getcwd
from os.path import isfile, join
import re


## Importamos el corpus
Lo primero que hacemos es importar el corpus al notebook para que podamos utilizarlos. En este caso definimos dos formas de cargar el corpus, ya sea por documento o cargando todos los documentos del folder.

In [36]:

# Obtenemos el path del folder donde se almacenan los corpus
folder_path = (getcwd() + r"/CorpusDocs")

# Los almacenamos en una lista donde se almacenan los nombres de los archivos.
# Esto es en caso de que usemos todos los corpus.
corpus_name_list = [f for f in listdir(folder_path) if isfile(join(folder_path, f))]

def loadAllCorpus():
    """
    Esta funcion carga todos los corpus que estan en el folder Corpus Docs.
    """
    corpis = ''
    for file in corpus_name_list:
        with open("./CorpusDocs/" + file, 'r', encoding="utf8") as f:
            corpis += f.read()
    return corpis

def loadCorpus(corpus_name):
    """
    Esta funcion nos sirve para cargar un corpus especifico
    """
    with open("./CorpusDocs/" + corpus_name, 'r', encoding="utf8") as f:
        corpus = f.read()
    return corpus
        

In [37]:
# Cargamos el corpus.

#corpus = loadAllCorpus()
corpus = loadCorpus('corpusML.txt')

## Limpieza del Texto
Separamos las palabras de las oraciones para poder trabajar con ellas individualmente

In [38]:
def add_eos_init(corpus):
    """
    Adds <eos> and <init> to a corpus. Based on line jumps
    """
    
    init = '<init> '
    eos = ' <eos>'
    corpus_in_eo = []


    corpus_in_eo = init + corpus + eos
    corpus_in_eo= corpus_in_eo.replace("\n", eos +" \n"+init)

    return corpus_in_eo

In [39]:
corpus_init = add_eos_init(corpus)
words = corpus_init.split()
print(words[:20])

['<init>', 'Comencé', 'a', 'trabajar', 'y', 'me', 'pegaron,', 'me', 'maltrataron', 'con', 'chicote', '<eos>', '<init>', 'Mis', 'patrones', 'me', 'pegaron', 'porque', 'no', 'me']


Eliminamos la puntuación del documento, acentos y normalizamos el texto en minusculas. Para hacer la eliminación de los símbolos de puntuación utilziamos una tabla de traducción para optimizar la velocidad de procesamiento. Tambien fue necesario extender la tabla de símbolos para que incluyera algunos símbolos latinos que faltaban.

Para eliminar acentos usamos la libreria unidecode que se tiene que instalar adicionalmente: `pip install unidecode`

In [40]:
import string
import unidecode

# Para poder mantener algunos flags quitamos < > de los elementos que se pueden eliminar
# de los simbolos de puntuación
punct = string.punctuation.replace("<", '')
punct = punct.replace(">", '')

print(punct)

!"#$%&'()*+,-./:;=?@[\]^_`{|}~


In [41]:
lat_punctuation = punct+'¿¡1234567890'
#print(lat_punctuation)

table = str.maketrans('', '', lat_punctuation)

In [75]:
clean_words = []

for word in words:
    word = word.lower()               # Minusculas
    word = unidecode.unidecode(word)  # Quitamos acentos.
    
    # Clean punctuation
    temp_w = []
    for letter in word:
        if letter not in lat_punctuation:
            temp_w.append(letter)
        word = ''.join(temp_w)

    clean_words.append(word)


True

## Stemming de Palabras
Para hacer el stemming de las palabras usamos NLTK. Para esto hay que instalar NLTK: 

Lo primero que hacemos es definir un stemmer. En este caso usaremos [Snowball Stemmer](http://snowball.tartarus.org/texts/introduction.html).

In [44]:
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('spanish')

In [45]:
stemmed_text = []
for word in clean_words:
    stemmed_text.append(stemmer.stem(word))
    
print(stemmed_text[:10])

['<init>', 'comenc', 'a', 'trabaj', 'y', 'me', 'pegaron,', 'me', 'maltrat', 'con']


## Bigramas
Para obtener los biogramas existentes del corpus creamos una fucnion que nos sierve para obtener todos los bigramas que existen en el corpus y a traves de el generar una lista de ellos.

In [46]:
def create_ngrams(stemmed_text, n):
    """
    Creates an n-gram structure from a stemmed or tokenized text. 
    
    Params
    ------
    stemmed_text: Tokens or stemmed words of a corpus
    n: the size of the n gram ex. 2 for a bigram
    """
    return zip(*[stemmed_text[i:] for i in range(n)])

In [47]:
bigramas = list(create_ngrams(stemmed_text, 2))

In [48]:
print(bigramas[0])

('<init>', 'comenc')


In [49]:
import collections
counter=collections.Counter(bigramas)       
    

In [50]:
print(counter)

Counter({('<eos>', '<init>'): 1704, ('<init>', 'pues'): 172, ('entonc', '<eos>'): 81, ('<init>', 'se'): 79, ('<init>', 'me'): 77, ('otra', 'vez'): 69, ('<init>', 'y'): 66, ('<init>', 'per'): 64, ('<init>', 'no'): 57, ('nomas', '<eos>'): 43, ('<init>', 'el'): 43, ('<init>', 'despues'): 42, ('par', 'que'): 38, ('<init>', 'entonc'): 36, ('cas', '<eos>'): 33, ('<init>', 'lueg'): 31, ('<init>', 'cuand'): 31, ('<init>', 'com'): 31, ('en', 'la'): 30, ('a', 'la'): 27, ('com', '<eos>'): 26, ('pues', 'se'): 26, ('pues', 'me'): 26, ('<init>', 'lo'): 26, ('<init>', 'por'): 25, ('<init>', 'ya'): 25, ('<init>', 'le'): 25, ('en', 'el'): 24, ('<init>', 'este…'): 23, ('<init>', 'la'): 23, ('per', 'pues'): 23, ('trabaj', '<eos>'): 22, ('fue', 'a'): 21, ('que', 'se'): 21, ('pues', 'entonc'): 21, ('<init>', 'hab'): 21, ('iba', 'a'): 20, ('<init>', 'he'): 19, ('no', 'se'): 19, ('ya', 'no'): 19, ('pues', 'asi'): 18, ('de', 'la'): 18, ('no', 'me'): 18, ('y', 'pues'): 18, ('much', '<eos>'): 18, ('<init>', 'si

In [51]:
print(counter['<init>', 'a'])

5


In [52]:
# Obtenemos el alfabeto de las palabras del stem
#alfabeto = set(stemmed_text)

alfabetoPI = []
for stem in stemmed_text:
    if stem not in alfabetoPI:
        alfabetoPI.append(stem)
        

alfabetoPI.remove('<init>')

print("AlfabetoPI total: {0}".format(len(alfabetoPI)))


AlfabetoPI total: 1594


## Visualización de Matrices
Utilizamos Pandas para visualizar la matriz A y la matriz Pi

In [53]:
import pandas as pd

In [56]:
MatrixA = pd.DataFrame(index=alfabetoPI, columns=alfabetoPI)

In [57]:
for x in alfabetoPI:
    for y in alfabetoPI:
        MatrixA.set_value(y,x, counter[y,x])

In [58]:
MatrixA

,comenc,a,trabaj,y,me,"pegaron,",maltrat,con,chicot,<eos>,...,calent,pajuel,vapor,quemart,"pasa,","quemas,","falta,",conform,"vecinas,",cai
comenc,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
a,0,0,13,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
trabaj,0,2,0,3,0,0,0,0,0,22,...,0,0,0,0,0,0,0,0,0,0
y,0,8,1,0,12,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
me,0,0,0,0,0,1,2,0,0,0,...,0,0,0,0,0,0,0,0,0,1
"pegaron,",0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
maltrat,0,0,0,0,0,0,0,1,0,2,...,0,0,0,0,0,0,0,0,0,0
con,0,0,3,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
chicot,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
<eos>,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [60]:

MatrixPI = pd.DataFrame(index=alfabetoPI, columns=['<init>'])

In [61]:
for y in alfabetoPI:
    if y is not '<eos>':
        MatrixPI.set_value(y,'<init>', counter['<init>',y])

In [62]:
MatrixPI

,<init>
comenc,1
a,5
trabaj,1
y,66
me,77
"pegaron,",0
maltrat,0
con,9
chicot,0
<eos>,1


# Realizado por:
- Bustos Ramírez Luis Enrique
- Flores Cortés Juan Pablo


In [63]:
for punc in lat_punctuation:
    print(punc)

!
"
#
$
%
&
'
(
)
*
+
,
-
.
/
:
;
=
?
@
[
\
]
^
_
`
{
|
}
~
¿
¡
1
2
3
4
5
6
7
8
9
0
